In [1]:
import os

In [2]:
%pwd

'd:\\AI-Projects\\Chicken-diseases-classification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\AI-Projects\\Chicken-diseases-classification'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    trainig_data_path: Path
    params_epochs: int
    params_batch_size: int
    params_is_augments: bool
    params_image_size: list

@dataclass(frozen=True)
class PrepareCallbackConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path




In [6]:
from ChickenDiseaseClassifier.constants import  *
from ChickenDiseaseClassifier.utils.common import  read_yaml,create_directories


In [7]:


class ConfigurationManager:
    def __init__(
        
        self,
        config_filepath= CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH
    ):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_prepare_callback_config(self)->PrepareCallbackConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir= os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensor_root_log_dir)
        ])

        prepare_callback_config= PrepareCallbackConfig(
            root_dir= Path(config.root_dir),
            tensorboard_root_log_dir= Path(config.tensor_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config
    

    def get_training_config(self)->TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params= self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir,"Chicken-facel-images")
        create_directories([
            Path(training.root_dir),
      
        ])

        training_config=TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.model_path),
            updated_base_model_path=Path(prepare_base_model.update_base_model_path),
            trainig_data_path=Path(training_data),
            params_epochs=params.EPOCES,
            params_batch_size=params.BATCH_SIZE,
            params_is_augments=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config



In [8]:
import os
import urllib.request as request
import zipfile
from ChickenDiseaseClassifier import logger
import tensorflow as tf 
import time


[2024-11-19 17:19:55,245:INFO:utils:NumExpr defaulting to 8 threads.]


In [9]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbackConfig):
        self.config = config
        
    @property
    def __create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    
    @property
    def __create_ckpt_callbacks(self):
        filepath = Path(self.config.checkpoint_model_filepath).with_suffix('.keras')
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=str(filepath),
            save_best_only=True,
    )

    
    def get_tb_ckpt_callbacks(self):
        return [self.__create_tb_callbacks, self.__create_ckpt_callbacks]
                                  



In [16]:
class Training:
    def __init__(self,config: TrainingConfig):
        self.config = config
    def get_base_modle(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    def train_valid_generator(self):
        datagenerators_kwrgs = dict(
            rescale = 1./255,
            validation_split = 0.20
        ) 

        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerators_kwrgs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory = self.config.trainig_data_path,
            subset = "validation",
            shuffle = True,
            **dataflow_kwargs
        )

        if self.config.params_is_augments:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                horizontal_flip=True,
                **datagenerators_kwrgs
            ) 
        else:
            train_datagenerator = valid_datagenerator

        

        self.train_generator = train_datagenerator.flow_from_directory(
            directory = self.config.trainig_data_path,
            subset = "training",
            shuffle = True,
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path: Path,model:tf.keras.Model):
        model.save(path)

    def train(self,callback_list:list):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator,
            callbacks=callback_list
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )



        


    

In [18]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_modle()
    training.train_valid_generator()
    training.train(
        callback_list=callback_list
    )
    
except Exception as e:
    raise e

[2024-11-19 17:23:32,993:INFO:common:Successfully loaded YAML file: config\config.yaml]
[2024-11-19 17:23:32,999:INFO:common:Successfully loaded YAML file: params.yaml]
[2024-11-19 17:23:33,002:INFO:common:Directory created: artifacts]
[2024-11-19 17:23:33,004:INFO:common:Directory created: artifacts\prepare_callbacks\checkpoint_dir]
[2024-11-19 17:23:33,006:INFO:common:Directory created: artifacts\prepare_callbacks\tensor_log_dir]
[2024-11-19 17:23:33,009:INFO:common:Directory created: artifacts\training]


AttributeError: 'Training' object has no attribute 'get_base_model'

In [ ]:

print(os.path.exists('artifacts\\data_ingestion\\Chicken-fecel-images'))

True


In [ ]:
print(os.path.exists('artifacts\data_ingestion\Chicken-fecel-images'))

True
